#StackOverflow assistant bot

Here we construct a chat bot that helps answer questions by redirecting us to stack overflow pages and also maintains a dialogue with the user!

For a chit-chat mode we will use a pre-trained neural network engine available from [ChatterBot](https://github.com/gunthercox/ChatterBot).


### Data description

To detect *intent* of users questions we will need two text collections:
- `tagged_posts.tsv` — StackOverflow posts, tagged with one programming language (*positive samples*).
- `dialogues.tsv` — dialogue phrases from movie subtitles (*negative samples*).


In [ ]:
def setup_starspace():
    if not os.path.exists("/usr/local/bin/starspace"):
        os.system("wget https://dl.bintray.com/boostorg/release/1.63.0/source/boost_1_63_0.zip")
        os.system("unzip boost_1_63_0.zip && mv boost_1_63_0 /usr/local/bin")
        os.system("git clone https://github.com/facebookresearch/Starspace.git")
        os.system("cd Starspace && make && cp -Rf starspace /usr/local/bin")
from setup.download_utils import download_project_resources
setup_starspace()
download_project_resources()

import sys
sys.path.append("..")
download_project_resources()

In [ ]:
from utils import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Intent Recognition


We want to write a bot, which will not only **answer programming-related questions**, but also will be able to **maintain a dialogue**. We would also like to detect the *intent* of the user from the question.

In [ ]:
import numpy as np
import pandas as pd
import pickle
import re

from sklearn.feature_extraction.text import TfidfVectorizer

### Data preparation

In [ ]:
def tfidf_features(X_train, X_test, vectorizer_path):
    """Performs TF-IDF transformation and dumps the model."""
 
    tfidf_vectorizer = TfidfVectorizer(min_df=5, max_df=0.9, ngram_range=(1, 2), token_pattern="(\S+)" )

    X_train = tfidf_vectorizer.fit_transform(X_train)
    X_test = tfidf_vectorizer.transform(X_test)

    with open(vectorizer_path, 'wb') as file:
      pickle.dump(tfidf_vectorizer, file)
    
    return X_train, X_test

In [ ]:
sample_size = 200000

dialogue_df = pd.read_csv('data/dialogues.tsv', sep='\t').sample(sample_size, random_state=0)
stackoverflow_df = pd.read_csv('data/tagged_posts.tsv', sep='\t').sample(sample_size, random_state=0)

Check how the data look like:

In [ ]:
dialogue_df.head()

,text,tag
82925,"Donna, you are a muffin.",dialogue
48774,He was here last night till about two o'clock....,dialogue
55394,"All right, then make an appointment with her s...",dialogue
90806,"Hey, what is this-an interview? We're supposed...",dialogue
107758,Yeah. He's just a friend of mine I was trying ...,dialogue


In [ ]:
stackoverflow_df.head()

,post_id,title,tag
2168983,43837842,Efficient Algorithm to compose valid expressio...,python
1084095,15747223,Why does this basic thread program fail with C...,c_cpp
1049020,15189594,Link to scroll to top not working,javascript
200466,3273927,Is it possible to implement ping on windows ph...,c#
1200249,17684551,GLSL normal mapping issue,c_cpp


We apply text_prepare for cleaning out the texts and titles of punctuation and other unwanted symbols.

In [ ]:
from utils import text_prepare

In [ ]:
dialogue_df['text'] = dialogue_df['text'].apply(text_prepare)
stackoverflow_df['title'] = stackoverflow_df['title'].apply(text_prepare)

### Intent recognition

We will do a binary classification on TF-IDF representations of texts. Labels will be either `dialogue` for general questions or `stackoverflow` for programming-related questions. First, prepare the data for this task:
- concatenate `dialogue` and `stackoverflow` examples into one sample
- split it into train and test in proportion 9:1, use *random_state=0* for reproducibility
- transform it into TF-IDF features

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = np.concatenate([dialogue_df['text'].values, stackoverflow_df['title'].values])
y = ['dialogue'] * dialogue_df.shape[0] + ['stackoverflow'] * stackoverflow_df.shape[0]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=0)
print('Train size = {}, test size = {}'.format(len(X_train), len(X_test)))

X_train_tfidf, X_test_tfidf = tfidf_features(X_train, X_test, './tfidf_vectorizer.pkl') 

Train size = 360000, test size = 40000


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
intent_recognizer = LogisticRegression(solver='newton-cg', C=10, random_state=0)
intent_recognizer.fit(X_train_tfidf, y_train)

LogisticRegression(C=10, random_state=0, solver='newton-cg')

In [ ]:
# Check test accuracy.
y_test_pred = intent_recognizer.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_test_pred)
print('Test accuracy = {}'.format(test_accuracy))

Test accuracy = 0.991575


Dump the classifier to use it in the running bot.

In [ ]:
pickle.dump(intent_recognizer, open(RESOURCE_PATH['INTENT_RECOGNIZER'], 'wb'))

### Programming language classification 

We also need to predict the programming language related to the question, for which we will use Logistic Regression with TF-IDF features.

In [ ]:
X = stackoverflow_df['title'].values
y = stackoverflow_df['tag'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print('Train size = {}, test size = {}'.format(len(X_train), len(X_test)))

Train size = 160000, test size = 40000


In [ ]:
vectorizer = pickle.load(open(RESOURCE_PATH['TFIDF_VECTORIZER'], 'rb'))

X_train_tfidf, X_test_tfidf = vectorizer.transform(X_train), vectorizer.transform(X_test)

In [ ]:
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
tag_classifier = OneVsRestClassifier(LogisticRegression(solver='newton-cg', penalty='l2', C=5, random_state=0))
tag_classifier.fit(X_train_tfidf, y_train)

OneVsRestClassifier(estimator=LogisticRegression(C=5, random_state=0,
                                                 solver='newton-cg'))

In [ ]:
# Check test accuracy.
y_test_pred = tag_classifier.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_test_pred)
print('Test accuracy = {}'.format(test_accuracy))

Test accuracy = 0.8007


Dump the classifier to use it in the running bot.

In [ ]:
pickle.dump(tag_classifier, open(RESOURCE_PATH['TAG_CLASSIFIER'], 'wb'))

## Part II. Ranking  questions with embeddings

To find a relevant answer (a thread from StackOverflow) on a question we use vector representations to calculate similarity between the question and existing threads.
However, it would be costly to compute such a representation for all possible answers in case the bot is running in online mode. For this reason, we create a database with pre-computed representations. These representations will be arranged by non-overlaping tags (programming languages), so that the search of the answer can be performed only within one tag each time. This will make our bot even more efficient and allow not to store all the database in RAM. 

We load StarSpace embeddings which were trained on Stack Overflow posts. These embeddings were trained in *supervised mode* for duplicates detection on the same corpus that is used in search. We can account on that these representations will allow us to find closely related answers for a question. 

In [ ]:
starspace_embeddings, embeddings_dim = load_embeddings('data/word_embeddings.tsv')
posts_df = pd.read_csv('data/tagged_posts.tsv', sep='\t')
counts_by_tag = posts_df.groupby(posts_df['tag']).count()
counts_by_tag = posts_df['tag'].value_counts().to_dict()

Now for each `tag` we create two data structures, which will serve as online search index:
* `tag_post_ids` — a list of post_ids with shape `(counts_by_tag[tag],)`. It will be needed to show the title and link to the thread;
* `tag_vectors` — a matrix with shape `(counts_by_tag[tag], embeddings_dim)` where embeddings for each answer are stored.


In [ ]:
import os
os.makedirs(RESOURCE_PATH['THREAD_EMBEDDINGS_FOLDER'], exist_ok=True)

for tag, count in counts_by_tag.items():
    tag_posts = posts_df[posts_df['tag'] == tag]
    
    tag_post_ids = tag_posts['post_id'].values
    
    tag_vectors = np.zeros((count, embeddings_dim), dtype=np.float32)
    for i, title in enumerate(tag_posts['title']):
        tag_vectors[i, :] = question_to_vec(title,starspace_embeddings, embeddings_dim)

    filename = os.path.join(RESOURCE_PATH['THREAD_EMBEDDINGS_FOLDER'], os.path.normpath('%s.pkl' % tag))
    pickle.dump((tag_post_ids, tag_vectors), open(filename, 'wb'))

##Chatting With The Bot

In [ ]:
!pip install chatterbot
!pip install chatterbot-corpus
!pip3 install pyyaml

from dialoge_manager import *

Now we are ready to test our chat bot! Let's chat with the bot and ask it some questions. Check that the answers are reasonable.

In [ ]:
questions = [
    "Hey", 
    "How are you doing?", 
    "What's your hobby?", 
    "How to write a loop in python?",
    "How to delete rows in pandas?",
    "python3 re",
    "What is the difference between c and c++",
    "Multithreading in Java",
    "Catch exceptions C++",
    "What is AI?",
]

for question in questions:
    answer = generate_answer(question)
    print('Q: %s\nA: %s \n' % (question, answer))

No value for search_text was available on the provided input


Q: Hey
A: hey 

Q: How are you doing?
A: hey 

Q: What's your hobby?
A: hey 

Q: How to write a loop in python?
A: I think its about python
 This thread might help you: https://stackoverflow.com/questions/14784623 

Q: How to delete rows in pandas?
A: I think its about python
 This thread might help you: https://stackoverflow.com/questions/40872090 

Q: python3 re
A: I think its about python
 This thread might help you: https://stackoverflow.com/questions/10769394 

Q: What is the difference between c and c++
A: I think its about c_cpp
 This thread might help you: https://stackoverflow.com/questions/25180069 

Q: Multithreading in Java
A: I think its about java
 This thread might help you: https://stackoverflow.com/questions/5692521 

Q: Catch exceptions C++
A: I think its about c_cpp
 This thread might help you: https://stackoverflow.com/questions/618215 

Q: What is AI?
A: I think its about java
 This thread might help you: https://stackoverflow.com/questions/30362773 

